In [255]:
import pandas as pd

In [256]:
log_file = "log_01-2022-04-17-21-43-20.csv"
log_df = pd.read_csv(f"LOG_CSV\{log_file}", sep=";", index_col=False)
log_df.reset_index()
log_df.set_index('date', inplace=True)
log_df.index = pd.to_datetime(log_df.index)
log_df.columns

Index(['log_type', 'symbol', 'open', 'high', 'low', 'close', 'volume',
       'order_ref', 'order_name', 'order_type', 'order_status', 'order_price',
       'order_size', 'order_value', 'order_commission', 'max_hold_date',
       'EMA200', 'EMA20', 'ATR', 'SuperTrend'],
      dtype='object')

In [257]:
filter_symbol    = log_df["symbol"] == "ORCL"
filter_buy       = log_df["order_type"] == "Buy"
filter_completed = log_df["order_status"] == "Completed"
filter_accepted  = log_df["order_status"] == "Accepted"
filter_canceled  = log_df["order_status"] == "Canceled"
market_order = log_df["order_name"] == "Market"

#log_df.loc[filter_symbol & (filter_completed | filter_canceled)].head(50)
log_df.loc[filter_symbol & (filter_accepted | filter_completed ) ].head(10)

,log_type,symbol,open,high,low,close,volume,order_ref,order_name,order_type,order_status,order_price,order_size,order_value,order_commission,max_hold_date,EMA200,EMA20,ATR,SuperTrend
date,,,,,,,,,,,,,,,,,,,,
2021-01-06,NOTIFY_ORDER,ORCL,62.27,63.35,61.91,62.57,8685400,4.0,Market,Buy,Accepted,62.96,7.63,0.00,0.00,2021-01-25,57.36,62.85,1.17,61.89
2021-01-06,NOTIFY_ORDER,ORCL,62.27,63.35,61.91,62.57,8685400,5.0,Stop,Sell,Accepted,60.67,-7.63,0.00,0.00,2021-01-25,57.36,62.85,1.17,61.89
2021-01-06,NOTIFY_ORDER,ORCL,62.27,63.35,61.91,62.57,8685400,6.0,Limit,Sell,Accepted,66.47,-7.63,0.00,0.00,2021-01-25,57.36,62.85,1.17,61.89
2021-01-06,NOTIFY_ORDER,ORCL,62.27,63.35,61.91,62.57,8685400,4.0,Market,Buy,Completed,62.27,7.63,475.10,0.48,2021-01-25,57.36,62.85,1.17,61.89
2021-01-22,NOTIFY_ORDER,ORCL,60.94,61.15,59.74,60.36,9896700,5.0,Stop,Sell,Completed,60.67,-7.63,475.10,0.46,2021-01-25,57.84,62.10,1.04,63.50
2021-02-12,NOTIFY_ORDER,ORCL,62.32,63.12,62.31,63.08,6663400,11.0,Market,Buy,Accepted,62.70,7.76,0.00,0.00,2021-03-03,58.48,62.54,1.15,60.13
2021-02-12,NOTIFY_ORDER,ORCL,62.32,63.12,62.31,63.08,6663400,12.0,Stop,Sell,Accepted,60.36,-7.76,0.00,0.00,2021-03-03,58.48,62.54,1.15,60.13
2021-02-12,NOTIFY_ORDER,ORCL,62.32,63.12,62.31,63.08,6663400,13.0,Limit,Sell,Accepted,66.28,-7.76,0.00,0.00,2021-03-03,58.48,62.54,1.15,60.13
2021-02-12,NOTIFY_ORDER,ORCL,62.32,63.12,62.31,63.08,6663400,11.0,Market,Buy,Completed,62.32,7.76,483.87,0.48,2021-03-03,58.48,62.54,1.15,60.13


In [258]:
def trades_statistics(trades):
    stats = dict()
    stats['symbol'] = trades.iloc[0]['symbol']
    stats['number_of_trades'] = len(trades)
    stats['gain'] = trades['net'].sum()
    
    return stats

In [259]:
def analyse_trades(symbol, log_df):
    filter_symbol = log_df["symbol"] == symbol
    filter_completed = log_df["order_status"] == "Completed"
    transactions = log_df.loc[filter_symbol & filter_completed]

    t_id = 0
    trades = []
    for i in range(0, len(transactions) - (len(transactions) % 2), 2):
        t_buy  = transactions.iloc[i].copy()
        t_sell = transactions.iloc[i+1].copy()
        days_held = (transactions.index[i+1] - transactions.index[i]).days

        trade = dict()
        trade['trade_id'] = t_id
        trade['symbol'] = t_buy['symbol']
        trade['days_held'] = days_held
        trade['shares'] = t_buy['order_size']
        
        trade['buy_date']   = transactions.index[i]
        trade['buy_order_ref'] = t_buy['order_ref']
        trade['buy_name'] = t_buy['order_name']
        trade['buy_price'] = t_buy['order_price']
        trade['buy_shares'] = t_buy['order_size']
        trade['buy_order_value'] = t_buy['order_value']
        trade['buy_tx_value'] = t_buy['order_size'] * t_buy['order_price'] - t_buy['order_commission']
        
        trade['sell_date'] = transactions.index[i+1]
        trade['sell_order_ref'] = t_sell['order_ref']
        trade['sell_name'] = t_sell['order_name']
        trade['sell_price'] = t_sell['order_price']
        trade['sell_shares'] = t_sell['order_size']
        trade['sell_order_value'] = t_sell['order_value']
        trade['sell_tx_value'] = (t_sell['order_size'] * t_sell['order_price'] * -1) - t_sell['order_commission']

        trade['net'] = trade['sell_tx_value'] - trade['buy_tx_value']

        t_id += 1
        trades.append(trade)
        #buy = dict()
        #buy['date'] = 
        #print(f"[{transactions.index[i]}] {t_buy['symbol']} -> BUY: {t_buy['order_name']} {t_buy['order_type']} | [{transactions.index[i+1]}] {t_sell['symbol']} -> SELL: {t_sell['order_name']} {t_sell['order_type']}")

    trades_df = pd.DataFrame(trades)

    analysis = dict()
    analysis['trades'] = trades_df
    analysis['stats'] = trades_statistics(trades_df)

    return analysis

In [260]:
all_symbols = list(set(log_df['symbol'].to_list()))
results = []
for symbol in all_symbols:
    result = analyse_trades(symbol, log_df)
    results.append(result)

[stats['stats'] for stats in results]


[{'symbol': 'ORCL', 'number_of_trades': 9, 'gain': 121.5523},
 {'symbol': 'AAPL', 'number_of_trades': 10, 'gain': 37.33240000000029}]

In [263]:
results[0]['trades']

,trade_id,symbol,days_held,shares,buy_date,buy_order_ref,buy_name,buy_price,buy_shares,buy_order_value,buy_tx_value,sell_date,sell_order_ref,sell_name,sell_price,sell_shares,sell_order_value,sell_tx_value,net
0,0,ORCL,16,7.63,2021-01-06,4.0,Market,62.27,7.63,475.10,474.6401,2021-01-22,5.0,Stop,60.67,-7.63,475.10,462.4521,-12.1880
1,1,ORCL,13,7.76,2021-02-12,11.0,Market,62.32,7.76,483.87,483.1232,2021-02-25,13.0,Limit,66.28,-7.76,483.87,513.8228,30.6996
2,2,ORCL,20,9.10,2021-03-12,15.0,Market,66.96,9.10,609.12,608.7260,2021-04-01,18.0,Market,70.46,-9.10,609.12,640.5460,31.8200
3,3,ORCL,9,8.26,2021-04-26,19.0,Market,74.97,8.26,619.33,618.6322,2021-05-05,21.0,Limit,79.70,-8.26,619.33,657.6620,39.0298
4,4,ORCL,20,8.08,2021-05-13,25.0,Market,77.53,8.08,626.57,625.8124,2021-06-02,28.0,Market,79.70,-8.08,626.57,643.3360,17.5236
5,5,ORCL,20,7.06,2021-08-19,33.0,Market,88.29,7.06,623.08,622.7074,2021-09-08,39.0,Market,88.14,-7.06,623.08,621.6484,-1.0590
6,6,ORCL,5,7.10,2021-09-09,40.0,Market,89.60,7.10,635.91,635.5200,2021-09-14,41.0,Stop,86.19,-7.10,635.91,611.3390,-24.1810
7,7,ORCL,7,7.17,2021-10-01,46.0,Market,87.76,7.17,629.21,628.6092,2021-10-08,48.0,Limit,93.01,-7.17,629.21,666.2117,37.6025
8,8,ORCL,21,5.36,2021-11-02,52.0,Market,93.73,5.36,502.64,501.8928,2021-11-23,55.0,Market,94.16,-5.36,502.64,504.1976,2.3048


In [261]:
df1 = results[0]['trades']
net_col = df1['net']
net_col.sum()


121.5523